---

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re

from ai_mas_hse_project.src.create_dataset import (
    parse_problem_from_url,
    parse_problem,
    save_to_json,
    parse_problem_from_file,
)

In [2]:
parse_problem_from_file(filename="../data/html_source/30346.html", main_topic="Алгебра")

{'id': 30346,
 'topic': 'Алгебра',
 'subtopic': 'Перестановки и подстановки (прочее)',
 'complexity_level': '2',
 'problem': '<p></p><p>На танцплощадке собрались <i>N</i> юношей и <i>N</i> девушек. Сколькими способами они могут разбиться на пары для участия в очередном танце?</p>\n<br/>',
 'solution': '',
 'answer': '<p></p><p><i>N</i>! способами. </p>'}

In [3]:
from IPython.core.display import HTML

HTML(
    "<p></p><p> \xa0 Подстановкой в уравнение убеждаемся в том, что \xa0<i>а</i> ≠ 0\xa0 и \xa0<i>а</i> ≠ ±1. <br/>\n \xa0 Из условия следует, что \xa0<i>а</i><sup>11</sup> + <i>а</i><sup>7</sup> + <i>а</i><sup>3</sup> = 1.\xa0 Умножив обе части этого равенства на \xa0<i>а</i><sup>4</sup> – 1,\xa0 получим \xa0<i>а</i><sup>15</sup> – <i>а</i><sup>3</sup> = <i>а</i><sup>4</sup> – 1\xa0 ⇔ \xa0<i>a</i><sup>4</sup> + <i>a</i><sup>3</sup> = <i>a</i><sup>15</sup> + 1.\xa0 Сравнив полученное равенство с равенством, данным в условии, получим \xa0<i>a<sup>n</sup> = a</i><sup>15</sup>.\xa0 Так как <i>a</i> по модулю отлично от нуля и единицы, то \xa0<i>n</i> = 15.</p>\n<br/>"
)

# Создание пустых HTML-файлов
Алгоритм создания выборки на основе задач с сайта problems.ru:
* Создаем пустые html-файлы в директории `ai_mas_hse_project/data/html_source`, где `название_файла = номер_задачи_на_сайте_problems_ru`
* Вручную заполняем данные болванки html-кодом. Вы спросите, а почему ты не сделал это в автоматическом режиме? Дело в том, что когда я пытался так сделать меня забанили за DDOS. Я не стал рисковать второй раз и отобрал задачи вручную.
* Парсим html-файлы и выполняем минимальную предобработку распарсенных строк.
* На выходе мы имеем список словарей, где каждый словарь - это информация по одной задаче с сайте problems.ru

In [4]:
# Алгебра и арифметика. Задачи ниже взяты отсюда https://problems.ru/view_by_subject_new.php?parent=88
list_algarith = [
    87981,
    87992,
    88081,
    88105,
    88142,
    104122,
    88129,
    88246,
    89919,
    30262,
    30289,
    30832,
    30862,
    30937,
    30941,
    31235,
    32830,
    32838,
    35012,
    88078,
    116023,
    116445,
    116528,
    116534,
    35676,
    34848,  # Здесь есть степень. Необходимо посмотреть как выглядит степень в распарсеном виде.
    34938,
    60652,
    64822,
    76501,  # Здесь очень много степеней. Посмотреть что они корректно отображаются в распарсенном виде.
    115968,  # Здесь очень много степеней. Посмотреть что они корректно отображаются в распарсенном виде.
    61199,  # Тригонометрическая задача. Тут можно применить вольфрам-альфа.
    116563,
    103822,  # Задача на дроби. Посмотреть корректность отображения
    107706,
    111895,
    61219,  # Тригонометрия
    64960,
    61222,  # Тригонометрическое уравнение без ответа и решения. Ответ можно написать самому здесь. Здесь можно применить вольфрам-альфа.
    78008,  # Тригонометрическое уравнение, где можно заюзать вольфрам-альфа.
    115444,  # Тригонометрическое уравнение. Для проверки ответа можно использовать вольфрам-альфа
    116883,  # Тригонометрия. Здесь нет решения, но есть ответ. Ответ можно проверить через вольфрам-альфа. РИСУНОК НА ПОЛ-РЕШЕНИЯ
    111261,  # Многочлены
    65636,
    86483,
    35641,  # Есть степени
    109027,  # Система уравнений
    65425,  # С дробями
    60976,  # С дробями
    116797,  # С дробями
    67400,  # С дробями
    61221,  # Задача без решения и без ответа, но ее можно решить через вольфрам-альфа
    104102,  # Система тригонометрических уравнений
    65985,  # Тригонометрия. Тут в решении есть рисунок. Его можно ручками заменить.
    64824,
    64948,
    116684,
    116875,
    35532,
    64670,
    105103,
    111262,  # С дробями
    65987,
    78159,
    66360,
    105085,
    35306,
    64671,  # Тут есть специфические latex символы. Необходимо посмотреть нормально ли они отрисовались
    116439,  # ТУТ ЕСТЬ РЕШЕНИЕ 1 И РЕШЕНИЕ 2. НУЖНО РУЧКАМИ ВПИСАТЬ ТОЛЬКО РЕШЕНИЕ 1 !!!!!!!!!!
    76430,
    65965,
]

# Комбинаторика. Задачи ниже взяты отсюда https://problems.ru/view_by_subject_new.php?parent=188
list_combinatorics = [
    60370,  # Задача без решения, но с ответом
    30320,  # Задача без решения, но с ответом
    30322,  # Задача без решения, но с ответом
    30321,  # Задача без решения, но с ответом
    30346,  # Задача без решения, но с ответом
    30419,
    30431,
    31083,
    60336,
    60381,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    30341,
    30722,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    60378,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    60382,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    60402,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    30690,  # ЗАДАЧА У КОТОРОЙ В РЕШЕНИИ РИСУНОК, НО МОЖНО ВРУЧНУЮ ЗАПИСАТЬ РЕШЕНИЕ
    30697,
    30699,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    30325,
    102999,
    116799,
    65344,
    32052,
    35129,
    35479,
    60345,
    35734,
    60450,  # ЗАДАЧА БЕЗ РЕШЕНИЯ, НО С ОТВЕТОМ
    35399,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    60384,
    60403,  # ЗАДАЧА У КОТОРОЙ В ОТВЕТЕ РИСУНОК, НО МОЖНО ВРУЧНУЮ НАПИСАТЬ ОТВЕТ
    111357,
    116990,  # ЗАДАЧА У КОТОРОЙ В РЕШЕНИИ РИСУНОК, НО МОЖНО ВРУЧНУЮ ЗАПИСАТЬ РЕШЕНИЕ
    109483,
    30778,
    35071,
    66576,
    108978,
    109436,  # С дробями
    111644,
    115450,
    116885,  # ЗАДАЧА У КОТОРОЙ В РЕШЕНИИ РИСУНОК, НО МОЖНО ВРУЧНУЮ ЗАПИСАТЬ РЕШЕНИЕ
    64511,
    64833,
    66720,
    98470,
]

# Теория вероятностей и статистика. Задачи ниже взяты отсюда https://problems.ru/view_by_subject_new.php?parent=208
list_probstat = [
    66038,  # ЗАДАЧА БЕЗ РЕШЕНИЯ, НО С ОТВЕТОМ
    65330,
    66034,  # ЗАДАЧА БЕЗ РЕШЕНИЯ, НО С ОТВЕТОМ
    65312,
    65265,  # С дробями
    65267,
    65333,
    65785,
    35352,
    35178,
    65315,
    65768,
]

# Математический анализ. Задачи ниже взяты отсюда https://problems.ru/view_by_subject_new.php?parent=265
list_mathan = [
    64891,
    34872,
    98427,
    66354,
    61213,  # У данной задачи есть только условие. Решения и ответа нет. Однако его можно получить через вольфрам-альфа.
]

# Олимпиадные задачи. Задачи ниже взяты отсюда https://problems.ru/view_by_source_new.php?parent=164898
list_olimpiada = [
    67443,
    67455,
    76414,
    105072,
    116226,
    30364,
    76429,
    77964,
    78025,
    78726,
    105197,
    78489,
    105120,
]

print(
    "Количество уникальных задач:",
    len(
        list(set(list_algarith + list_combinatorics + list_probstat + list_mathan + list_olimpiada))
    ),
)

Количество уникальных задач: 147


In [5]:
parse_problem_from_file(filename="../data/html_source/30690.html", main_topic="Алгебра")

{'id': 30690,
 'topic': 'Алгебра',
 'subtopic': 'Сочетания и размещения',
 'complexity_level': '2+',
 'problem': '<p></p><p>У одного школьника есть 6 книг по математике, а у другого – 8.  Сколькими способами они могут обменять три книги одного на три книги другого?</p><br/>',
 'solution': '<p></p><p>Первый школьник может выбрать 3 книги для обмена <img align="MIDDLE" border="0" height="36" src="show_document.php?id=89917" width="21"/> способами, второй – <img align="MIDDLE" border="0" height="36" src="show_document.php?id=89918" width="21"/> способами.  Таким образом, число возможных обменов равно <img align="MIDDLE" border="0" height="36" src="show_document.php?id=89919" width="107"/>.</p><br/>',
 'answer': '<p></p><p>1120 способами.</p>'}

In [6]:
list_algarith = list(set(list_algarith))
list_combinatorics = list(set(list_combinatorics))
list_probstat = list(set(list_probstat))
list_mathan = list(set(list_mathan))
list_olimpiada = list(set(list_olimpiada))

In [7]:
###====================Данная ячейка создает пустые HTML-файлы, где НАЗВАНИЕ=НОМЕР_ЗАДАЧИ====================###

# for item in list_all:
#     filename = f"../data/html_delete/{item}.html"
#     with open(filename, 'w', encoding='utf-8') as f:
#         f.write("")  # Пустой файл

#     print(f"Создан пустой файл: {filename}")

# Парсинг HTML-файлов. Создание итогового датасета

In [19]:
def remove_author_block(text: str):
    """
    Удаляет блок с информацией об авторе из текста
    """
    # Паттерн для удаления всего div с автором
    pattern = r'<div class="catalogueproblemauthorold">.*?</div>'

    # Удаляем все вхождения
    cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL)

    return cleaned_text


def replace_str_with_img_to_nan(text: str):
    if "<img" in text:
        return ""


def replace_anchors_to_nan(text):
    """
    Заменяет все вхождения <a name="..."></a> на пробел
    """
    # Паттерн для якорных ссылок
    pattern = r'<a\s+name="[^"]*"\s*></a>'

    # Заменяем на пробел
    return re.sub(pattern, "", text)

In [20]:
def clean_parse_html(x: str):
    """Удаляем из строки ненужные html-теги и прочий мусор, который не влияет на интерпретацию"""
    # Заменяем HTML-сущности
    replacements = {
        "\xa0": " ",
        "<p>": " ",
        "</p>": " ",
        "<i>": " ",
        "</i>": " ",
        "<br/>": " ",
        "\n": " ",
        '<p align="justify">': " ",
        # '<a name="8.24"></a>': ' ',
        # '<a name="8.37"></a>': ' ',
    }

    for old, new in replacements.items():
        x = x.replace(old, new)

    x = remove_author_block(x)

    x = re.sub(r"\s+", " ", x).strip()

    x = replace_anchors_to_nan(x)

    return x

In [21]:
replace_answer_dict = {
    60381: "C_{n}^{2}",
    30722: "C_{30}^{8} способами",
    60378: "A_{9}^{4} = 9 * 8 * 7 * 6 = 3024 способами",
    60382: "C_{n}^{3}",
    60402: "C_{10}^{6} = C_{10}^{4} = 210 чисел",
    35399: "C_{20}^{10} = 184756 способов",
    60403: "C_{m+1}^{n} способами",
    30699: "C_{3}^{7} = 35 способами",
}

replace_solution_dict = {
    30690: "Первый школьник может выбрать 3 книги для обмена C_{6}^{3} способами, второй - C_{3}^{8} способами. Таким образом, число возможных обменов равно C_{6}^{3} * C_{3}^{8} = 1120.",
    116990: "Треугольник однозначно определяется тремя своими вершинами. Из двадцати точек три можно выбрать C_{20}^{3} = 1140  способами. Учитывая, что три точки, лежащие на одной стороне десятиугольника, не образуют треугольника, получим:  1140 - 10 = 1130 треугольников.",
    116885: "Для каждых двух вершин девятиугольника существует ровно одна его вершина, равноудаленная от них, поэтому каждый из получающихся равнобедренных, но не равносторонних, треугольников однозначно определяется своим основанием. Количество способов выбрать две точки из девяти равно С_{9}^{2}. Но при таком способе подсчета каждый из трёх равносторонних треугольников учтён три раза. Поэтому искомое число равно 36 - 6 = 30.",
}

In [22]:
dict_map = {
    "algarith": {"main_topic_name": "Алгебра и арифметика", "list_tasks": list_algarith},
    "combinatorics": {"main_topic_name": "Комбинаторика", "list_tasks": list_combinatorics},
    "probstat": {"main_topic_name": "Вероятность и статистика", "list_tasks": list_probstat},
    "mathan": {"main_topic_name": "Математический анализ", "list_tasks": list_mathan},
    "olimpiada": {"main_topic_name": "Олимпиадные задачи", "list_tasks": list_olimpiada},
    # "logictheory": "Логика и теория множеств",
    # "geometry": "Геометрия"
}

In [ ]:
res_list = []

for i_key, i_val in dict_map.items():
    for i_task in i_val["list_tasks"]:
        dict_task = parse_problem_from_file(
            filename=f"../data/html_source/{i_task}.html", main_topic=i_val["main_topic_name"]
        )
        if i_task in replace_answer_dict:
            dict_task["answer"] = replace_answer_dict[i_task]

        if i_task in replace_solution_dict:
            dict_task["solution"] = replace_solution_dict[i_task]

        dict_task["problem"] = clean_parse_html(dict_task["problem"])
        dict_task["solution"] = clean_parse_html(dict_task["solution"])
        dict_task["answer"] = clean_parse_html(dict_task["answer"])
        dict_task["answer"] = dict_task["answer"].rstrip(". ")

        res_list.append(dict_task)

In [24]:
res_list

[{'id': 115968,
  'topic': 'Алгебра и арифметика',
  'subtopic': 'Исследование квадратного трехчлена',
  'complexity_level': '2+',
  'problem': 'Известно, что разность кубов корней квадратного уравнения ax ² + bx + c = 0 равна 2011. Сколько корней имеет уравнение ax ² + 2 bx + 4 c = 0?',
  'solution': 'Из условия следует, что первое уравнение имеет два корня. Следовательно, его дискриминант D = b ² – 4 ac &gt; 0. Упрощенный дискриминант второго уравнения также равен b ² – 4 ac , значит, оно также имеет два корня.',
  'answer': 'Два корня'},
 {'id': 64824,
  'topic': 'Алгебра и арифметика',
  'subtopic': 'Текстовые задачи (прочее)',
  'complexity_level': '3',
  'problem': 'Три пирата нашли клад, состоящий из 240 золотых слитков общей стоимостью 360 долларов. Стоимость каждого слитка известна и выражается целым числом долларов. Может ли оказаться так, что добычу нельзя разделить между пиратами поровну, не переплавляя слитки?',
  'solution': 'Пусть один из слитков стоит 121 доллар, а кажд

In [47]:
dict_task = parse_problem_from_file(
    filename=f"../data/html_source/116883.html", main_topic=i_val["main_topic_name"]
)
dict_task["problem"] = clean_parse_html(dict_task["problem"])
dict_task["solution"] = clean_parse_html(dict_task["solution"])
dict_task["answer"] = clean_parse_html(dict_task["answer"])

In [48]:
dict_task

{'id': 116883,
 'topic': 'Олимпиадные задачи',
 'subtopic': 'Числовые неравенства. Сравнения чисел',
 'complexity_level': '3',
 'problem': 'Сравните: sin 3 и sin 3°.',
 'solution': '<img align="middle" src="show_document.php?id=1708703"/>',
 'answer': 'sin 3 &gt; sin 3°.'}

# Удаление ненужных тегов из распарсенного HTML

In [33]:
def clean_parse_html(x: str):
    """Удаляем из строки ненужные html-теги и прочий мусор, который не влияет на интерпретацию"""
    # Заменяем HTML-сущности
    replacements = {
        "\xa0": " ",  # неразрывный пробел
        "<p>": "",
        "</p>": "",
        "<i>": "",
        "</i>": "",
        "<br/>": "",
        "\n": "",
    }

    for old, new in replacements.items():
        x = x.replace(old, new)

    x = re.sub(r"\s+", " ", x).strip()

    return x

In [34]:
test_str = parse_problem_from_file(filename="../data/html_source/64671.html", main_topic="Алгебра")

In [35]:
test_str

{'id': 64671,
 'topic': 'Алгебра',
 'subtopic': 'Алгебраические уравнения и системы уравнений (прочее)',
 'complexity_level': '3+',
 'problem': '<p></p><p>Число <i>a</i> – корень уравнения \xa0<i>х</i><sup>11</sup> + <i>х</i><sup>7</sup> + <i>х</i><sup>3</sup> = 1.\xa0 При каких натуральных значениях <i>n</i> выполняется равенство \xa0<i>a</i><sup>4</sup> + <i>a</i><sup>3</sup> = <i>a<sup>n</sup></i> + 1?</p>\n<br/>',
 'solution': '<p></p><p> \xa0 Подстановкой в уравнение убеждаемся в том, что \xa0<i>а</i> ≠ 0\xa0 и \xa0<i>а</i> ≠ ±1. <br/>\n \xa0 Из условия следует, что \xa0<i>а</i><sup>11</sup> + <i>а</i><sup>7</sup> + <i>а</i><sup>3</sup> = 1.\xa0 Умножив обе части этого равенства на \xa0<i>а</i><sup>4</sup> – 1,\xa0 получим \xa0<i>а</i><sup>15</sup> – <i>а</i><sup>3</sup> = <i>а</i><sup>4</sup> – 1\xa0 ⇔ \xa0<i>a</i><sup>4</sup> + <i>a</i><sup>3</sup> = <i>a</i><sup>15</sup> + 1.\xa0 Сравнив полученное равенство с равенством, данным в условии, получим \xa0<i>a<sup>n</sup> = a</i><s

In [36]:
test_str["problem"] = clean_parse_html(test_str["problem"])
test_str["solution"] = clean_parse_html(test_str["solution"])
test_str["answer"] = clean_parse_html(test_str["answer"])

In [37]:
test_str

{'id': 64671,
 'topic': 'Алгебра',
 'subtopic': 'Алгебраические уравнения и системы уравнений (прочее)',
 'complexity_level': '3+',
 'problem': 'Число a – корень уравнения х<sup>11</sup> + х<sup>7</sup> + х<sup>3</sup> = 1. При каких натуральных значениях n выполняется равенство a<sup>4</sup> + a<sup>3</sup> = a<sup>n</sup> + 1?',
 'solution': 'Подстановкой в уравнение убеждаемся в том, что а ≠ 0 и а ≠ ±1. Из условия следует, что а<sup>11</sup> + а<sup>7</sup> + а<sup>3</sup> = 1. Умножив обе части этого равенства на а<sup>4</sup> – 1, получим а<sup>15</sup> – а<sup>3</sup> = а<sup>4</sup> – 1 ⇔ a<sup>4</sup> + a<sup>3</sup> = a<sup>15</sup> + 1. Сравнив полученное равенство с равенством, данным в условии, получим a<sup>n</sup> = a<sup>15</sup>. Так как a по модулю отлично от нуля и единицы, то n = 15.',
 'answer': 'При n = 15.'}

In [39]:
from IPython.core.display import HTML

HTML(test_str["problem"])

In [42]:
from IPython.core.display import HTML

HTML(
    """<a name="8.24"></a> Вычислите следующие произведения: а) <!-- MATH $\\sin20^{\\circ}\\sin40^{\\circ}\\sin60^{\\circ}\\sin80^{\\circ}$ --> sin 20<sup><tt>o</tt></sup>sin 40<sup><tt>o</tt></sup>sin 60<sup><tt>o</tt></sup>sin 80<sup><tt>o</tt></sup>; б) <!-- MATH $\\cos20^{\\circ}\\cos40^{\\circ}\\cos60^{\\circ}\\cos80^{\\circ}$ --> cos 20<sup><tt>o</tt></sup>cos 40<sup><tt>o</tt></sup>cos 60<sup><tt>o</tt></sup>cos 80<sup><tt>o</tt></sup>."""
)

# Парсинг HTML через GET-запросы
Из-за этого подхода меня забанили на сайте problems.ru за DDOS

In [1]:
# import json

# from ai_mas_hse_project.src.create_dataset import parse_problem_from_url

In [2]:
# # Пример URL (замените на реальный)
# problem_url = "https://problems.ru/view_problem_details_new.php?id=87981"

# result = parse_problem_from_url(problem_url)

# # if result:
# #     print("Успешно спарсены данные:")
# #     print(json.dumps(result, ensure_ascii=False, indent=2))
# # else:
# #     print("Не удалось спарсить данные.")